# 赤検知の仕方
## 赤の閾値を決める:場所ごとに異なる方が良いかもしれない


In [60]:
#定数エリア
RED_THRE= 100 # 赤判定の閾値
RED_PART_THRE = 0.5 #エリア内の赤存在判定の割合

#インポートエリア
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import glob

#jupyter用
%matplotlib inline

## 画像をRGB値の行列として見る


In [72]:
from pathlib import Path, PurePosixPath
file = Path("../images/IMG_4493.jpeg").resolve()
img = cv2.imread(str(file))
# plt.imshow(img)
from find_color import find_rect_of_target_color
rects = find_rect_of_target_color(img,3)

In [71]:

rects

(1565, 806, 59, 55)

In [68]:

if len(rects) > 0:
    i = 1
    for rect in rects:
    # rect = max(rects, key=(lambda x: x[2] * x[3]))

        if rect[3] > 5 and rect[2] > 5: # if red circle is one

            cv2.rectangle(img, tuple(rect[0:2]), tuple(rect[0:2] + rect[2:4]), (0, 0, 255), thickness=2)
            cv2.putText(img,
                        text=str(i),
                        org=(rect[0], rect[1]),
                        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                        fontScale=4,
                        color=(255, 0, 0),
                        thickness=2,
                        lineType=cv2.LINE_4)
            print(rect)
            i += 1

TypeError: 'int' object is not subscriptable

In [67]:
# plt.imshow(img)
%whos

Variable                    Type           Data/Info
----------------------------------------------------
Path                        type           <class 'pathlib.Path'>
PurePosixPath               type           <class 'pathlib.PurePosixPath'>
RED_PART_THRE               float          0.5
RED_THRE                    int            100
cv2                         module         <module 'cv2' from 'c:\\U<...>kages\\cv2\\__init__.py'>
file                        WindowsPath    C:\Users\user\detect_red\<...>idth\images\IMG_4493.jpeg
find_rect_of_target_color   function       <function find_rect_of_ta<...>or at 0x000001BF6C33E5E0>
glob                        module         <module 'glob' from 'C:\\<...>n2kfra8p0\\lib\\glob.py'>
i                           int            1
img                         ndarray        3024x4032x3: 36578304 elems, type `uint8`, 36578304 bytes (34.8837890625 Mb)
np                          module         <module 'numpy' from 'c:\<...>ges\\numpy\\__init__.py'>

In [ ]:
# cv2.putText(img,"11111111",(10,10),cv2.FONT_HERSHEY_SIMPLEX,1.5,(255,255,255),1, cv2.LINE_AA)
# plt.imshow(img)
cv2.imwrite('output.jpg',img)

## Rが赤の閾値以上の時は255, G=0, B=0 で赤の二値化をする


## 赤存在判定の閾値を決める：エリア内に～ピクセル以上